In [1]:
import argparse, os, pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score

# Huggingface:
from datasets import Dataset
from transformers import AutoTokenizer, EsmForSequenceClassification as ESMClf, EsmModel as ESM
from transformers import TrainingArguments, Trainer

In [2]:
def main(path):
    tokenizer = AutoTokenizer.from_pretrained(path)
    model = ESMClf.from_pretrained(path)

    pstr = "MKEEEISNKKDKGISRRKFLGGAAATAAAFTIVPRHVLGGAGYTPPSEKLNVAGVGVGGMGGENIINVAGMERDKDRNLIKKREGENIVALCDVDEKFASDIFNAFHKAKKYKDFRKMLEKQ"
    t_pstr = tokenizer(pstr, return_tensors='pt')

    out = model(**t_pstr, output_attentions = True)

    attns = out.attentions

    # How to access:
    print('len pstr', len(pstr))
    print('len attns', len(attns))
    print('attn', attns[0].shape)

In [3]:
path = "/lustre/isaac/scratch/oqueen/DeepSurface/esm/model_outputs/esm_sp=0_35m/checkpoint-4000"

tokenizer = AutoTokenizer.from_pretrained(path)
model = ESMClf.from_pretrained(path)

In [4]:
pstr = "MKEEEISNKKDKGISRRKFLGGAAATAAAFTIVPRHVLGGAGYTPPSEKLNVAGVGVGGMGGENIINVAGMERDKDRNLIKKREGENIVALCDVDEKFASDIFNAFHKAKKYKDFRKMLEKQ"
t_pstr = tokenizer(pstr, return_tensors='pt')

out = model(**t_pstr, output_attentions = True)
attns = out.attentions

In [5]:
tokens = tokenizer.convert_ids_to_tokens(t_pstr['input_ids'][0])

In [6]:
#model_view(attns, )
import torch

for i, a in enumerate(attns):
    a = a.detach().clone().cpu()
    print(attns[i].device)

torch.save((attns, tokens), 'example.pt')

cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu


In [18]:
from transformers_interpret import SequenceClassificationExplainer
model.eval()
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)

In [19]:
for param in model.parameters():
    param.requires_grad = False

In [21]:
print(dir(tokenizer))

['SPECIAL_TOKENS_ATTRIBUTES', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_tokens', '_additional_special_tokens', '_auto_class', '_batch_encode_plus', '_batch_prepare_for_model', '_bos_token', '_call_one', '_cls_token', '_convert_id_to_token', '_convert_token_to_id', '_convert_token_to_id_with_added_voc', '_create_repo', '_create_trie', '_decode', '_decode_use_source_tokenizer', '_encode_plus', '_eos_token', '_eventual_warn_about_too_long_sequence', '_eventually_correct_t5_max_length', '_from_pretrained', '_get_files_timestamps', '_get_padding_truncation_strategies', '_id_to_token', '_in_target_context_manager', '_mask_token', '_pad', '_pad_toke

In [20]:
with torch.no_grad():
    word_attributions = cls_explainer("MKEEEISNKKDKGISRRKFLGGAAATAAAFTIVPRHVLGGAGYTPPSEKLNVAGVGVGGMGGENIINVAGMERDKDRNLIKKREGENIVALCDVDEKFASDIFNAFHKAKKYKDFRKMLEKQ")

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.